In [ ]:
import pandas as pd
import numpy as np
import os
import glob
from config import load_config
paths = load_config(dataset_key='all')
from natsort import natsorted
from skimage import io
import matplotlib.pyplot as plt
import cv2 as cv
import h5py

In [ ]:
# Read metadata files and extract timestamp of each frame
metadata_file = glob.glob(os.path.join(paths['patchcord_autofluo'],'*.csv'))
print(metadata_file)
metadata = pd.read_csv(metadata_file[0])
timestamp = metadata.CameraTimestampSeconds + 10**-6*metadata.CameraTimestampMicroSeconds
time_vector = timestamp - timestamp[0]

framestamp = np.asarray(metadata.Framestamp)
skip_frames = np.asarray(np.where(np.diff(framestamp) > 1))
skip_frames = skip_frames.flatten() +1
print(skip_frames)

f, ax = plt.subplots(figsize=(6,2))
ax.plot(np.diff(time_vector))
ax.set(xlabel='Framestamp', ylabel='Timestep')
plt.show()

f, ax = plt.subplots(figsize=(6,2))
ax.plot(np.diff(framestamp))
ax.set(xlabel='Framestamp', ylabel='Timestep')
plt.show()

width = metadata.Width[0]
print(width)
Xoffset = metadata.XOffset[0]
print(Xoffset)
height = metadata.Height[0]
print(height)
Yoffset = metadata.YOffset[0]
print(Yoffset)

In [ ]:
# Read tiff files and extract intensity at each frame and pixel
files = os.listdir(paths['patchcord_autofluo'])
print(files)
tiff_dir = os.path.join(paths['patchcord_autofluo'],files[1])
tiff_files = os.listdir(tiff_dir)
tiff_files = np.array(natsorted(tiff_files))
tiff_files

In [ ]:
# Unskew image, rotation followed by affine transformation

# Copied from running preprocess_01_unskewimage
theta_r = -1.1508764781734035
pt1 = [920-Xoffset, 583-Yoffset] 
pt2 = [895-Xoffset, 691-Yoffset] 
pt3 = [1096-Xoffset, 588-Yoffset] 
pt4 = [895-Xoffset, 583-Yoffset] 
pt5 = [895-Xoffset, 691-Yoffset] 
pt6 = [1071-Xoffset, 588-Yoffset]
fiber1_location = [630-Yoffset,590-Yoffset]
fiber2_location = [685-Yoffset,645-Yoffset]

# Load data and unskew
rows,cols = [height, width]
M1 = cv.getRotationMatrix2D(((cols-1)/2.0,(rows-1)/2.0),theta_r,1)
pts1 = np.float32([pt1, pt2, pt3])
pts2 = np.float32([pt4, pt5, pt6])
M2 = cv.getAffineTransform(pts1,pts2)

for seq in range(0,np.size(tiff_files,0)):
    print(seq)
    tiff_file_path = os.path.join(tiff_dir,tiff_files[seq])
    temp = io.imread(tiff_file_path).astype(float)
    for frame in range(0,np.size(temp,0)):
        temp_rotated = cv.warpAffine(temp[frame,:,:],M1,(cols,rows))
        img = cv.warpAffine(temp_rotated,M2,(cols,rows))
        fiber1_m = np.mean(img[fiber1_location[1]:fiber1_location[0],:],axis=0)
        fiber2_m = np.mean(img[fiber2_location[1]:fiber2_location[0],:],axis=0)
        fiber1_m = np.expand_dims(fiber1_m, axis=0)
        fiber2_m = np.expand_dims(fiber2_m, axis=0)
        if ((seq==0) and (frame==0)):
            fiber1 = fiber1_m
            fiber2 = fiber2_m
        else:
            fiber1 = np.append(fiber1, fiber1_m, axis=0)
            fiber2 = np.append(fiber2, fiber2_m, axis=0)

print(fiber1.shape)
print(fiber2.shape)

In [ ]:
# Interleave signals into individual lasers
num_lasers = 3
if skip_frames.size == 0:
    img_new = fiber1
else:
    # Insert nan in skipped frames
    for skip in skip_frames:
        img_new = np.insert(fiber1,skip,np.nan,axis=0)

ch1 = img_new[0::3,:]
ch2 = img_new[1::3,:]
ch3 = img_new[2::3,:]
ch1.shape

In [ ]:
# Save pre-processed data hdf5
# To save: time vector, wavelength, intensity

cut = np.min([ch1.shape[0],ch2.shape[0],ch3.shape[0]])
time = time_vector[1::3]
time = time[0:cut].to_numpy()
print(time)

c = pd.read_hdf(paths['root']/'pixel_to_nm.hdf5', key='Camera_pixel', more='r')
w = pd.read_hdf(paths['root']/'pixel_to_nm.hdf5', key='Wavelength_nm', more='r')
wavelength = w.to_numpy()
camera_px = c.to_numpy()

Channel1 = np.zeros([cut,wavelength.shape[0]]).astype('int')
Channel2 = np.zeros([cut,wavelength.shape[0]]).astype('int')
Channel3 = np.zeros([cut,wavelength.shape[0]]).astype('int')
for px in range(0,wavelength.shape[0]):
    Channel1[:,px] = ch1[0:cut,camera_px[px]]
    Channel2[:,px] = ch2[0:cut,camera_px[px]]
    Channel3[:,px] = ch3[0:cut,camera_px[px]]

# Take average autofluorescence
Channel1_mean = np.mean(Channel1,axis=0)
Channel2_mean = np.mean(Channel2,axis=0)
Channel3_mean = np.mean(Channel3,axis=0)

print(Channel1_mean.shape)
f,ax = plt.subplots(figsize=(6,2))
ax.plot(Channel1_mean, linewidth=1)
plt.show()

data_preprocessed = {'Time':time, 'Wavelength':wavelength, 'Channel1':Channel1_mean, 'Channel2':Channel2_mean, 'Channel3':Channel3_mean}
for key in data_preprocessed.keys():
    print(f'\n{key}')
    print(data_preprocessed[key])

hf = h5py.File(paths['patchcord_autofluo'] / 'patchcord_autofluo.hdf5','a')
for key in data_preprocessed.keys():
    hf.create_dataset(key, data = data_preprocessed[key])
hf.close()

In [ ]:
with h5py.File(paths['patchcord_autofluo']/'patchcord_autofluo.hdf5','r') as f:
    print(f.keys())

f = h5py.File(paths['patchcord_autofluo'] / 'patchcord_autofluo.hdf5','r')
channel_1 = f['Channel1']
print(channel_1.shape)
channel_2 = f['Channel2']
print(channel_1.shape)
channel_3 = f['Channel3']
print(channel_1.shape)
time = f['Time']
print(time.shape)
wavelength = f['Wavelength']
print(wavelength.shape)

print(time[-1])

In [ ]:
f,ax = plt.subplots(figsize=(6,2))
i = ax.imshow(np.transpose(Channel1), aspect='auto', vmin=0, vmax=900, extent=[np.min(time), np.max(time), 699, 400])
ax.set(xlabel='Time (sec)', ylabel='Wavelength', title='Channel 1')
ax.grid(False)
f.colorbar(i,ax=ax)
plt.show()

f,ax = plt.subplots(figsize=(6,2))
i = ax.imshow(np.transpose(Channel2), aspect='auto', vmin=0, vmax=800, extent=[np.min(time), np.max(time), 699, 400])
ax.set(xlabel='Time (sec)', ylabel='Wavelength', title='Channel 2')
ax.grid(False)
f.colorbar(i,ax=ax)
plt.show()

f,ax = plt.subplots(figsize=(6,2))
ax.plot(wavelength[:], Channel1[10,:])
ax.set(xlabel='Wavelength (nm)', ylabel='Intensity', title='Channel 1') #, ylim=[0,2000])
plt.show()

f,ax = plt.subplots(figsize=(6,2))
ax.plot(wavelength[:], Channel2[10,:])
ax.set(xlabel='Wavelength (nm)', ylabel='Intensity', title='Channel 2') #, ylim=[0,2000])
plt.show()

f,ax = plt.subplots(figsize=(6,2))
ax.plot(wavelength[:], Channel3[10,:])
ax.set(xlabel='Wavelength (nm)', ylabel='Intensity', title='Channel 3') #, ylim=[0,1000])
plt.show()

f,ax = plt.subplots(figsize=(6,2))
ax.plot(time[:], Channel1[:,530-400], linewidth=1)
ax.set(xlabel='Time (sec)', ylabel='Intensity', title='530nm')
plt.show()

f,ax = plt.subplots(figsize=(6,2))
ax.plot(time[:], Channel2[:,620-400], linewidth=1)
ax.set(xlabel='Time (sec)', ylabel='Intensity', title='620nm')
plt.show()

f,ax = plt.subplots(figsize=(6,2))
ax.plot(time[:], Channel3[:,450-400], linewidth=1)
ax.set(xlabel='Time (sec)', ylabel='Intensity', title='450nm')
plt.show()